# 1.部署yolov5
### 1. 仓库：https://github.com/ultralytics/yolov5
### 2. 从仓库 git clone ，经测试python3.6也能行，但是jetson nano性能不能训练，因此用电脑训练好将权重放入jetson nano
### 3. 按照官方教程，安装requirements.txt上的环境到jetson nano
### 4. 因为电脑要部署gpu，所以注释掉requirement.txt中的torch与torchvision
### 5. 电脑去英伟达官网下载cuda：https://developer.nvidia.com/cuda-downloads?target_os=Windows&target_arch=x86_64&target_version=10&target_type=exe_local
### 6. 电脑去pytorch按照选择生成安装命令：https://pytorch.org/get-started/locally/
### 7. 将安装命令在创建好的anaconda环境下执行（上述安装requirement也在环境下）
### 8. 在电脑中用 cd /d 地址 进入yolov5的下载地址，
### 9. 由于github被墙，建议修改utils下的downloads.py第77行加入镜像https://ghproxy.com/
### 10. 执行 python detect.py --source 0 查看是否运用到了了gpu
### 11. 增加虚拟内存大小，从而让workers=8时内存不会炸开
### 12. 执行python train.py --cfg models/yolov5s.yaml --batch-size -1 --data ../garbage/garbage.yaml (按照train.py内部459-523行对参数的定义来写)
### 13. 注意建立数据库的格式和增加数据库精度的方法
**增加精度：https://blog.csdn.net/asd123pwj/article/details/117171868**
**数据格式：
+ **|garbage**
+ **|**
+ **|----images**
+ **|--------train**
+ **|--------val**
+ **|**
+ **|----labels**
+ **|--------train**
+ **|--------val**
+ **|**
+ **|----garbage.yaml(格式参考coco128.yaml)**


# 小问题
### 1. vscode不能使用conda到虚拟环境：https://blog.csdn.net/a12355556/article/details/114398391
### 2. torch1.11会出现不兼容，降级为torch1.10
### 3. 训练时出现WARNING: Ignoring corrupted image：https://blog.csdn.net/vibration_xu/article/details/122580372

# 2. 将yolov5权重文件转换为engine（转换出来在jetson nano不可用）
https://github.com/wang-xinyu/tensorrtx/tree/master/yolov5
### 1. 在与yolov5同级目录下git clone
### 2. 点开tensorrtx下的yolov5文件夹
### 3. 在yololayer.h中修改识别类型数和输入图片宽高等
### 4. 在yolov5.cpp中可修改设备和batch等
### 5. 将tensorrtx/yolov5下的gen_wts.py copy到yolov5下
### 6. 在yolov5下运行python gen_wts.py -w (要转换的权重) -o ().wts
### 7. cd到tensorrtx/yolov5下，创建build并且进入
### 8. 将转换好的.wts文件放到build目录下
### 9. cmake..
### 10. make
### 11. 这时所在目录会生成一个叫yolov5的执行文件
### 12. sudo ./yolov5 -s ().wts ().engine [n/s/m/l/x/...]将由某个格式的权重转换为.wts文件的权重转换为.engine格式
### 13. 生成成功，可以用 sudo ./yolov5 -d ().engine path/to/images 测试

# 3. 使用yolov5的export.py转换.engine(转换出来依旧不可用)
### 1. 安装cuda，cudnn，tensorrt（记得安装它的python）
cudnn：https://zhuanlan.zhihu.com/p/99880204?from_voters_page=true https://docs.nvidia.com/deeplearning/cudnn/install-guide/index.html#prerequisites-windows
tensorrt：https://blog.csdn.net/dou3516/article/details/124188371
### 2. pip install onnx
### 3. python export.py --weights yolov5s.pt --include engine --imgsz  --device  --data 。。。。。看源码436-459
colab ：https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb#scrollTo=VTRwsvA9u7ln


# 4. 转换出来不能在nano使用的原因是因为电脑使用的是cuda11.6,cudnn8,tensorrt8，而nano版本不同导致转换好的.engine的版本序列号与nano不同，报错：[TensorRT] ERROR: 1: [stdArchiveReader.cpp::StdArchiveReader::34] Error Code 1: Serialization (Serialization assertion safeVersionRead == safeSerializationVersion failed.Version tag does not match. Note: Current Version: 43, Serialized Engine Version: 89)
实例：https://github.com/wang-xinyu/tensorrtx/issues/632
### 解决方法：在nano下转换，使用export.py， 具体参数根据源码436-459行进行修改。

# 5.Tips
### 1. 使用像素越高的图片进行训练，map越高，但是在nano运行速度越慢（因此寻求tensorrt加速）

# 6.目前进度
### 1. 完成yolov5环境配置，明确了detect.py与train.py和export.py参数的设置（电脑端在conda环境下）
### 2. 完成了cuda，cudnn，tensorrt环境的配置（电脑端）
### 3. 明确如何制作训练集与训练集的数据文件 .yaml
### 4. 收集了一定量的数据进行训练，试图谋求更高的map

# 7. 目标设想展望
### 1.可否调整优化器optimizer提高精度或者训练效果
### 2.多少个epochs可使map变得足够好
### 3.测试集有何作用？
### 4.

# 8. 还需补充的内容
### 1. 先找出源码中框选目标的代码，修改代码从而在识别时图像中标出框选的中心点
### 2. 解决jetson nano中导出的画面是由深度相机拍摄的，改成rgb相机便于观察
### 3. 将框选出的中心点给到realsense写的代码，输出中心点的【x,y,z】
### 4. 在ros中写一个tf坐标广播，用rviz实时演示坐标的变换

# 实验记录
### 1. imgz：1280 | model：yolov5s.yaml | weights: yolov5s.pt |hyp: low | opt : SGD | epochs: 20  | worker: 16
### 2. imgz:   640  | model:   yolov5s.yaml | weights: yolov5s.pt | hpy: low | opt : SGD | epochs: 20 | worker: 16
### 3. imgz:   640  | model:   yolov5s.yaml | weights: yolov5s.pt | hpy: med | opt : SGD | epochs: 20 | worker: 16 (快)
### 4. imgz:   640  | model:   yolov5s.yaml | weights: yolov5s.pt | hpy: high | opt : SGD | epochs: 20 | worker: 16
### 5. imgz:   640  | model:   yolov5s.yaml | weights: yolov5s.pt | hpy: low  | opt : Adam | epochs: 20 | worker: 16(不佳)
### 6. imgz:   640  | model:   yolov5s.yaml | weights: yolov5s.pt | hpy: med | opt : Adamw | epochs: 20 | worker: 16(不佳)
### 7.  imgz:   640  | model:   yolov5s.yaml | weights: yolov5s.pt | hpy: VOC | opt : SGD | epochs: 20 | worker: 16(快)（不佳）
### 8.  imgz:   640  | model:   yolov5s.yaml | weights: yolov5s.pt | hpy: Ob | opt : SGD | epochs: 20 | worker: 16(不佳)
### 9. imgz:   640  | model:   yolov5【m】.yaml | weights: yolov5s.pt | hpy: high | opt : SGD | epochs: 20 | worker: 16 （不佳）

## 最终筛选出 1，2，3，4号，进入测试环节
### 1. F1小，P波动大且小，R小，PR面积小 （排除）
### 2. F1平稳，can达到0.9左右，sb达到0.7左右有波动，b达到0.4左右波动较大较差，P波动相对较小，R相对较低较小，PR面积不错，混淆矩阵数值不行
### 3. F1相对平稳，sb波动衰减大，P波动较大特别是sb，R平稳数值不错，PR面积平稳sb与b面积小，混淆矩阵数值不行（排除）
### 4. F1最平稳数值最佳，P波动略大特别是b数值不佳，R平稳性好数值最佳，PR值较平稳sb有下降（选择）

# 综上，选择方案4！！！



# 绘制中心点，在E:\0_Project\yolov5\utils\plots.py （103-112）

# 9、在colab训练
### unzip 无法解压大于4gb的文件，选择用7z
### 7z x ().zip